<a href="https://colab.research.google.com/github/pk2971/Air-Quality-vs-Temperature-time-series/blob/main/Covid_19_image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import matplotlib

**COVID 19 image Classification**

Author: Praharshita Kaithepalli(pk2971@rit.edu)

Dataset found on Kaggle: https://www.kaggle.com/datasets/pranavraikokte/covid19-image-dataset

As a person whose lungs have been effected by COVID-19, me and m family had to submit the lung scan to the doctor and wait for his response to confirm if it was COVID 19 or just a viral fever. This was also a time where the COVID 19 cases were blowing up in India and we have difficulties finding a doctors appointment and hospital beds. Using AI/ML we can find a way to easily classify if a patient has COVID 19 or not based upon the scanning images.